In [3]:
# !pip install --upgrade --quiet llama-index-llms-openai llama-index-agent-openai 

In [7]:
import os
import json
from typing import Sequence, List

from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI

from api_keys import open_ai_key

import nest_asyncio

nest_asyncio.apply()

In [8]:
os.environ["OPENAI_API_KEY"] = open_ai_key

In [9]:
?ChatMessage

Init signature:
ChatMessage(
    *,
    role: llama_index.core.base.llms.types.MessageRole = <MessageRole.USER: 'user'>,
    content: Optional[Any] = '',
    additional_kwargs: dict = None,
) -> None
Docstring:      Chat message.
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           ~/miniconda3/envs/gpt/lib/python3.11/site-packages/llama_index/core/base/llms/types.py
Type:           ModelMetaclass
Subclasses:     

In [10]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)



def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

In [11]:
class YourOpenAIAgent:
    def __init__(
        self,
        tools: Sequence[BaseTool] = [],
        llm: OpenAI = OpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
        chat_history: List[ChatMessage] = [],
    ) -> None:
        self._llm = llm
        self._tools = {tool.metadata.name: tool for tool in tools}
        msg = """  
                you are a chat bot that wants to help students with their liner algebra problems. 
                You need to think step by step and explain the solution to the student as well. 
                Your task is complete only after student affirms that he/she has no more questions.
                always ask whether your response satisfies the user, and be creative while asking.
        """
        chat_history.append(ChatMessage(role="user", content=msg))
        self._chat_history = chat_history
        

    def reset(self) -> None:
        self._chat_history = []

    def chat(self, message: str) -> str:
        chat_history = self._chat_history
        chat_history.append(ChatMessage(role="user", content=message))
        tools = [
            tool.metadata.to_openai_tool() for _, tool in self._tools.items()
        ]

        ai_message = self._llm.chat(chat_history, tools=tools).message
        additional_kwargs = ai_message.additional_kwargs
        chat_history.append(ai_message)

        tool_calls = ai_message.additional_kwargs.get("tool_calls", None)
        # parallel function calling is now supported
        if tool_calls is not None:
            for tool_call in tool_calls:
                function_message = self._call_function(tool_call)
                chat_history.append(function_message)
                ai_message = self._llm.chat(chat_history).message
                chat_history.append(ai_message)

        return ai_message.content

    def _call_function(self, tool_call: dict) -> ChatMessage:
        id_ = tool_call["id"]
        function_call = tool_call["function"]
        tool = self._tools[function_call["name"]]
        output = tool(**json.loads(function_call["arguments"]))
        return ChatMessage(
            name=function_call["name"],
            content=str(output),
            role="tool",
            additional_kwargs={
                "tool_call_id": id_,
                "name": function_call["name"],
            },
        )

In [12]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = OpenAIAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)

In [13]:
HQ = "I have 20 bags and each bag has 40 apples. how many total apples do i have?"

response = agent.chat(HQ)
print(str(response))

Added user message to memory: I have 20 bags and each bag has 40 apples. how many total apples do i have?
=== Calling Function ===
Calling function: multiply with args: {
  "a": 20,
  "b": 40
}
Got output: 800

You have a total of 800 apples.


In [15]:
HQ = "what if I have only 300 apples and 5 bags, how many apples in each bag?"
print(str(agent.chat(HQ)))

Added user message to memory: what if I have only 300 apples and 5 bags, how many apples in each bag?


ValueError: Tool with name divide not found